# Thinking Like SQL in Pandas

This Notebook Describes the process of working with data in Pandas DataFrames while thinking in SQL

Having worked extensively with SQL, when I first started working with dataframes in Pandas, I was stuck thinking in SQL and wondering how to implement SQL commands in Pandas Data frames.

Below are a few most commonly used SQL commands implemented in Pandas DataFrames.

### CREATE TABLE

The 'CREATE TABLE' statement in SQL is used to create a new table in the database.

Below is its Implementation in SQL (Oracle):

> create table df (A varchar2(20), B varchar2(20), C number, D number); <br>

> insert into df (A, B, C, D)<br>
  > WITH names AS (<br>
  
	> SELECT 'foo', 'one', 1, 10	FROM dual UNION ALL<br>
	> SELECT 'bar', 'one', 2, 20	FROM dual UNION ALL<br>
	> SELECT 'foo', 'two', 3, 30	FROM dual UNION ALL<br>
	> SELECT 'bar', 'three', 4, 40  FROM dual<br>
  )<br>
  SELECT * FROM names<br>
;

Its Implementation in Pandas is as follows:

In [1]:
import pandas as pd
import numpy as np
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar'],
                   	'B' : ['one', 'one', 'two', 'three'],
                   	'C' : [1, 2, 3, 4],
                   	'D' : [10, 20, 30, 40]})
df

,A,B,C,D
0,foo,one,1,10
1,bar,one,2,20
2,foo,two,3,30
3,bar,three,4,40


Moving ahead, we explore a few more SQL commands' implementation in Pandas:

### WHERE CLAUSE

The SQL where clause is used to Filter records. Only the rows which fulfill the conditions in the where clause are let through. Following is its Implementation in SQL:

> select * from df where A = 'foo';

It's Implementation in Pandas is as follows:

In [2]:
df[df['A']=='foo']

,A,B,C,D
0,foo,one,1,10
2,foo,two,3,30


### ORDER BY

Order By sorts the rows in an ascending or descending order as per the column specified in the Order By Clause.

SQL:
> select * from df order by A, B;

Pandas:

In [3]:
df.sort_values(['A', 'B'])

,A,B,C,D
1,bar,one,2,20
3,bar,three,4,40
0,foo,one,1,10
2,foo,two,3,30


As seen in the output, the rows are first sorted by column A and then by column B.

P.S. Ascending is the default order in which the result is sorted when not specified otherwise.

To specify the sorting order for each column, the syntax is as follows: <br>
The below script will sort column A in descending and C in Ascending.

SQL:
> select * from df order by A desc, C;

Pandas:

In [4]:
df.sort_values(['A', 'C'], ascending=[False, True])

,A,B,C,D
0,foo,one,1,10
2,foo,two,3,30
1,bar,one,2,20
3,bar,three,4,40


### GROUP BY
Group by groups records into as per the values in the column specified.

The following example gives the number of rows in each group of values in column 'A'<br>
SQL Implementation:
> select A, count(*) from df group by A;

Pandas Implementation:

In [5]:
df.groupby('A').size()

A
bar    2
foo    2
dtype: int64

Now if we need to see the count of vslues in each column for the group of values in Column 'A'<br>
It's SQL Implementation is:
> select A, count(B), count(C), count(D) from df group by A;

And, in Pandas, its as follows:

In [6]:
df.groupby('A').count()

,B,C,D
A,,,
bar,2,2,2
foo,2,2,2


We can apply aggregate functions to the groups as follows: <br>

SQL:
> select A, sum(C), sum(D) from df group by A;

Pandas:

In [7]:
df.groupby('A').sum()

,C,D
A,,
bar,6,60
foo,4,40


### JOINS

Joins enable us to merge two or more tables to receive the records which satisfy the join criteria.

To demonstrate joins, we need one more dataframe and SQL table. The code for that is similar to the code seen in 'CREATE TABLE' section above. <br>

SQL:
> create table df1 (A number, B varchar2(20));<br>
insert into df1 (A, B)<br>
  WITH names AS (<br>
    SELECT 10, 'TEN'      FROM dual UNION ALL<br>
    SELECT 20, 'TWENTY'   FROM dual UNION ALL<br>
    SELECT 30, 'THIRTY'   FROM dual UNION ALL<br>
    SELECT 50, 'FIFTY'    FROM dual<br>
  )<br>
  SELECT * FROM names<br>
;<br>
SELECT * FROM DF1;

Pandas:

In [8]:
df1 = pd.DataFrame({'A': [10, 20, 30, 50], 'B': ['TEN', 'TWENTY', 'THIRTY', 'FIFTY']})
df1

,A,B
0,10,TEN
1,20,TWENTY
2,30,THIRTY
3,50,FIFTY


Quick Note: While SQL is case-insensitive, Python is case sensitive.

#### INNER JOIN

Returns records which satisfy the join criteria<br>

SQL:
> SELECT * FROM DF<br>
INNER JOIN DF1 ON DF.D = DF1.A<br>
;

Pandas:

In [9]:
pd.merge(df, df1, left_on='D', right_on='A')

,A_x,B_x,C,D,A_y,B_y
0,foo,one,1,10,10,TEN
1,bar,one,2,20,20,TWENTY
2,foo,two,3,30,30,THIRTY


#### LEFT JOIN

Returns all the rows in the left table and only the rows satisfying the join criteria from the right table <br>

SQL:
> SELECT * FROM DF<br>
LEFT JOIN DF1 ON DF.D = DF1.A<br>
;

Pandas:

In [10]:
pd.merge(df, df1, how='left', left_on='D', right_on='A')

,A_x,B_x,C,D,A_y,B_y
0,foo,one,1,10,10.0,TEN
1,bar,one,2,20,20.0,TWENTY
2,foo,two,3,30,30.0,THIRTY
3,bar,three,4,40,NaN,NaN


#### RIGHT JOIN

Returns the rows that satisfy the join in the left table and all the rows from the right table<br>

SQL:
> SELECT * FROM DF<br>
RIGHT JOIN DF1 ON DF.D = DF1.A<br>
;

Pandas:

In [11]:
pd.merge(df, df1, how='right', left_on='D', right_on='A')

,A_x,B_x,C,D,A_y,B_y
0,foo,one,1.0,10.0,10,TEN
1,bar,one,2.0,20.0,20,TWENTY
2,foo,two,3.0,30.0,30,THIRTY
3,NaN,NaN,NaN,NaN,50,FIFTY


#### FULL OUTER JOIN

Returns all the rows from both the tables <br>
The values are null for whichever row/column does not meet the join criteria<br>

SQL:
> SELECT * FROM DF<br>
FULL OUTER JOIN DF1 ON DF.D = DF1.A<br>
;

Pandas:

In [12]:
pd.merge(df, df1, how='outer', left_on='D', right_on='A')

,A_x,B_x,C,D,A_y,B_y
0,foo,one,1.0,10.0,10.0,TEN
1,bar,one,2.0,20.0,20.0,TWENTY
2,foo,two,3.0,30.0,30.0,THIRTY
3,bar,three,4.0,40.0,NaN,NaN
4,NaN,NaN,NaN,NaN,50.0,FIFTY


Quick Note: Indexes in SQL begin with 1 whereas in Python it begins with 0

As we see in the examples above the code in Pandas is relatively smaller to that in SQL.<br> Hopefully this Notebook helps you with your thinking in SQL while working with Pandas Data Frames.